<a href="https://colab.research.google.com/github/gksthdals/NLTK/blob/main/10.%20Analyzing_the_Meaning_of_Sentences_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" Main Topic

1. How can we represent natural language meaning so that a computer can process these representations?
2. How can we associate meaning representations with an unlimited set of sentences?
3. How can we use programs that connect the meaning representations of sentences to stores of knowledge?

"""

## 1. Natural Language Understanding

### Querying a Database

In [ ]:
# SQL : Structured Query Language

# SELECT Country FROM city_table WHERE City = 'athens'

In [ ]:
import nltk
nltk.download('book_grammars')

[nltk_data] Downloading package book_grammars to /root/nltk_data...
[nltk_data]   Unzipping grammars/book_grammars.zip.


True

In [ ]:
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [ ]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in China'

trees = list(cp.parse(query.split()))
for tree in trees:
  print(tree)
print()

answer = trees[0].label()['SEM']
print(answer)

# [SEM='']인 경우를 제외하고 문장을 만들기 위해서
answer = [s for s in answer if s]
print(answer)

q = ' '.join(answer)
print(q)

(S[SEM=(SELECT, City FROM city_table, WHERE, , , Country="china")]
  (NP[SEM=(SELECT, City FROM city_table)]
    (Det[SEM='SELECT'] What)
    (N[SEM='City FROM city_table'] cities))
  (VP[SEM=(, , Country="china")]
    (IV[SEM=''] are)
    (AP[SEM=(, Country="china")]
      (A[SEM=''] located)
      (PP[SEM=(, Country="china")]
        (P[SEM=''] in)
        (NP[SEM='Country="china"'] China)))))

(SELECT, City FROM city_table, WHERE, , , Country="china")
['SELECT', 'City FROM city_table', 'WHERE', 'Country="china"']
SELECT City FROM city_table WHERE Country="china"


In [ ]:
nltk.download('city_database')

[nltk_data] Downloading package city_database to /root/nltk_data...
[nltk_data]   Unzipping corpora/city_database.zip.


True

In [ ]:
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows: print(r[0], end=' ')

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

In [ ]:
# sql0.fcfg : translation from English to SQL

### Natural Language, Semantics and Logic

In [ ]:
# Declarative sentences are true or false in certain situations.
# Definite noun phrases and proper nouns refer to things in the world.

In [ ]:
"""
consistent : 여러 개의 문장이 동시에 성립 가능
inconsistent : 여러 개의 문장이 모순 관계

consistency and inconsistency
"""

## 2. Propositional Logic

In [ ]:
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


In [ ]:
read_expr = nltk.sem.Expression.fromstring

In [ ]:
read_expr('-(P & Q)')

<NegatedExpression -(P & Q)>

In [ ]:
read_expr('P & Q')

<AndExpression (P & Q)>

In [ ]:
read_expr('P | (R -> Q)')

<OrExpression (P | (R -> Q))>

In [ ]:
read_expr('P <-> -- P')

<IffExpression (P <-> --P)>

In [ ]:
lp = nltk.sem.Expression.fromstring
SnF = read_expr('SnF')
NotFnS = read_expr('-FnS')
R = read_expr('SnF -> -FnS')
prover = nltk.Prover9()
prover.prove(NotFnS, [SnF, R])

LookupError: ignored

In [ ]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])

In [ ]:
val['P']

True

In [ ]:
dom = set()
g = nltk.Assignment(dom)

In [ ]:
m = nltk.Model(dom, val)

In [ ]:
m.evaluate('(P & Q)', g)

True

In [ ]:
m.evaluate('-(P & Q)', g)

False

In [ ]:
m.evaluate('(P & R)', g)

False

In [ ]:
m.evaluate('(P | R)', g)

True

## 3. First-Order Logic

### Syntax

In [ ]:
"""
Unary/Binary Predicate

walk(angus) : unary predicate
love(margrietje, brunoke) : binary predicate

###

non-logical constants : 참/거짓을 판단할 수 없는 -> see, angus and bertie
logical constants : 참/거짓을 판단할 수 있는 -> boolean opr

###

e : the type of entities
t : the type of formulas (expression which have truth values)

"""